In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
import matplotlib as mpl

import matplotlib.pyplot as plt
import platform
if platform.system() == "Windows":
    plt.rcParams['font.family'] = ['Times new roman',] # windows->Heiti TC
else:
    plt.rcParams['font.family'] = ['Heiti TC',] # windows->Heiti TC
    
plt.rcParams['axes.unicode_minus'] = False # windows->simhei
plt.rcParams['figure.dpi'] = 200

import geopandas as gpd
import shapely
from shapely.geometry import Point,Polygon,LineString,MultiLineString,MultiPoint,MultiPolygon

import pickle
from tqdm import tqdm
from glob import glob
import joblib

import seaborn as sns

In [2]:
import os

import platform
if platform.system() == "Windows":
    os.chdir(r'H:\BaiduSyncdisk\DR.MENG-Full\Y2024-002-DT-NANJING-ACCESSIBILITY-MAUP')
else:
    os.chdir(r'/Volumes/SANDISK/DR.MENG')

In [3]:
grid_name = 'grid500' # community grid250 grid500 grid1000
trans_type = 'nav'   # walk cycle pt nav 

file_name = f'od_{trans_type}_{grid_name}_od' 
time_threshold = 900 # 900 1800

In [4]:
grid_types = [
    # 'buildings',
              'grid250', 'grid500',
              'grid1000', 'grid1500', 'grid2000',
              'grid2500', 'grid3000',
              'community'
             ]

# get pop

In [5]:
buildings_with_pop = gpd.read_file(r"./od_request_data_buildings/buildings_with_id_pop.geojson")

In [6]:
buildings_with_pop.head(1)

,building_id,省,市,县,乡,building_pop,geometry
0,b_0,江苏省,南京市,建邺区,沙洲街道,87,"POLYGON ((118.72926 31.99482, 118.72953 31.994..."


In [7]:
# buildings_with_pop.head(10).explore()

In [8]:
buildings_with_pop['geometry'] = buildings_with_pop['geometry'].representative_point()

In [9]:
# buildings_with_pop.head(10).explore()

In [10]:
grid_name

'grid500'

In [11]:
grid = gpd.read_file(f"./shp/use_bounds/nanjing_{grid_name}.shp")

In [12]:
grid.head(1)

,net_id,Id,geometry
0,11834,south,"POLYGON ((118.7796 31.87374, 118.77998 31.8782..."


In [13]:
result = gpd.sjoin(grid, buildings_with_pop, predicate='contains')

In [14]:
result

,net_id,Id,geometry,index_right,building_id,省,市,县,乡,building_pop
14,11925,south,"POLYGON ((118.80093 31.87691, 118.80131 31.881...",68678,b_126424,江苏省,南京市,江宁区,秣陵街道,177
14,11925,south,"POLYGON ((118.80093 31.87691, 118.80131 31.881...",68680,b_126426,江苏省,南京市,江宁区,秣陵街道,192
14,11925,south,"POLYGON ((118.80093 31.87691, 118.80131 31.881...",68675,b_126421,江苏省,南京市,江宁区,秣陵街道,174
14,11925,south,"POLYGON ((118.80093 31.87691, 118.80131 31.881...",68681,b_126427,江苏省,南京市,江宁区,秣陵街道,180
14,11925,south,"POLYGON ((118.80093 31.87691, 118.80131 31.881...",68674,b_126420,江苏省,南京市,江宁区,秣陵街道,164
...,...,...,...,...,...,...,...,...,...,...
3562,22971,north,"POLYGON ((118.74815 32.24858, 118.74853 32.253...",19298,b_87567,江苏省,南京市,六合区,葛塘街道,20
3562,22971,north,"POLYGON ((118.74815 32.24858, 118.74853 32.253...",19299,b_87568,江苏省,南京市,六合区,葛塘街道,22
3563,22972,north,"POLYGON ((118.75341 32.24825, 118.75379 32.252...",47457,b_72755,江苏省,南京市,六合区,大厂街道,189
3563,22972,north,"POLYGON ((118.75341 32.24825, 118.75379 32.252...",47456,b_72754,江苏省,南京市,六合区,大厂街道,190


In [15]:
resultx = result[['net_id','building_pop']].groupby(by='net_id').sum().reset_index() # net_id, 

In [16]:
resultx.head(1)

,net_id,building_pop
0,11925,1075


In [17]:
grid

,net_id,Id,geometry
0,11834,south,"POLYGON ((118.7796 31.87374, 118.77998 31.8782..."
1,11835,south,"POLYGON ((118.78484 31.87341, 118.78522 31.877..."
2,11836,south,"POLYGON ((118.79008 31.87309, 118.79046 31.877..."
3,11837,south,"POLYGON ((118.79531 31.87277, 118.79569 31.877..."
4,11838,south,"POLYGON ((118.80055 31.87244, 118.80093 31.876..."
...,...,...,...
3570,23050,north,"POLYGON ((118.74853 32.25305, 118.74891 32.257..."
3571,23051,north,"POLYGON ((118.75379 32.25272, 118.75417 32.257..."
3572,23130,north,"POLYGON ((118.74365 32.25784, 118.74404 32.262..."
3573,23131,north,"POLYGON ((118.74891 32.25751, 118.7493 32.2619..."


In [18]:
gridx = grid.merge(resultx,how='left').sort_values(by='building_pop')

In [19]:
# gridx[gridx['building_pop'] > 0]

In [20]:
gridx

,net_id,Id,geometry,building_pop
1930,20689,south,"POLYGON ((118.85187 32.10287, 118.85225 32.107...",0.0
2348,21241,south,"POLYGON ((118.95063 32.14608, 118.95102 32.150...",1.0
2382,21291,south,"POLYGON ((118.95102 32.15055, 118.95141 32.155...",2.0
1829,20468,south,"POLYGON ((118.79526 32.11986, 118.79564 32.124...",2.0
234,12936,south,"POLYGON ((118.85788 31.92729, 118.85827 31.931...",3.0
...,...,...,...,...
3570,23050,north,"POLYGON ((118.74853 32.25305, 118.74891 32.257...",NaN
3571,23051,north,"POLYGON ((118.75379 32.25272, 118.75417 32.257...",NaN
3572,23130,north,"POLYGON ((118.74365 32.25784, 118.74404 32.262...",NaN
3573,23131,north,"POLYGON ((118.74891 32.25751, 118.7493 32.2619...",NaN


# function： calculate accessibility

In [21]:
file_name

'od_nav_grid500_od'

In [22]:
od_nav = pd.read_csv(r'./data_to_cal_accessibility/{}.csv'.format(file_name))
od_nav.head()

,od_id,distance,duration,name,park_id,net_id,lng,lat,x_parks,y_parks
0,od_279718,7673.3,1606.85,月牙湖公园,p_23,19173,118.791507,31.983071,118.825728,32.031568
1,od_1118164,24484.0,5168.80,太子山公园,p_87,16806,118.629262,32.060390,118.749298,32.217740
2,od_1715412,17523.3,3824.80,中山植物园,p_131,17502,118.728666,32.112740,118.829339,32.052412
3,od_0,28063.7,5166.00,宝船厂遗址公园,p_0,11834,118.782410,31.875810,118.730990,32.063972
4,od_141009,14376.8,3798.55,窨子山文化遗址公园,p_14,20153,118.848279,32.091860,118.797008,31.991402


In [23]:
od_nav2 = pd.read_csv(r'./data_to_cal_accessibility/od_walk_community_od.csv') # community labe->XZQDM
od_nav2.head()

,od_id,distance,duration,name,park_id,XZQDM,lng,lat,x_parks,y_parks
0,od_106433,7907,6777,下马坊遗址公园,p_122,320104012001,118.771207,32.035421,118.839428,32.048234
1,od_0,18904,16203,宝船厂遗址公园,p_0,320116001000,118.744560,32.210293,118.730990,32.063972
2,od_19218,23858,20449,愚园,p_24,320113006008,118.916834,32.130622,118.768545,32.018494
3,od_86667,23856,20448,河西中央公园,p_99,320111003005,118.702921,32.177023,118.718034,31.999079
4,od_44649,29302,25116,江宁体育公园,p_54,320113001001,118.899847,32.158499,118.846486,31.949202


In [24]:
def get_gird_pop(grid_name, col_name):
    buildings_with_pop = gpd.read_file(r"./od_request_data_buildings/buildings_with_id_pop.geojson")
    buildings_with_pop['geometry'] = buildings_with_pop['geometry'].representative_point()
    
    grid = gpd.read_file(f"./shp/use_bounds/nanjing_{grid_name}.shp")
    result = gpd.sjoin(grid, buildings_with_pop, predicate='contains')
    resultx = result[[col_name,'building_pop']].groupby(by=col_name).sum().reset_index()
    print(resultx.head())
    print(grid.head())
    gridx = grid.merge(resultx,how='left').sort_values(by='building_pop')
    gridx = gridx[[col_name, 'building_pop']]
    gridx[col_name] = gridx[col_name].apply(lambda x: f'g_{x}')
    
    return gridx

In [25]:
get_gird_pop(grid_name, 'net_id')

   net_id  building_pop
0   11925          1075
1   11926          3905
2   12294           536
3   12297           777
4   12383          6030
   net_id     Id                                           geometry
0   11834  south  POLYGON ((118.7796 31.87374, 118.77998 31.8782...
1   11835  south  POLYGON ((118.78484 31.87341, 118.78522 31.877...
2   11836  south  POLYGON ((118.79008 31.87309, 118.79046 31.877...
3   11837  south  POLYGON ((118.79531 31.87277, 118.79569 31.877...
4   11838  south  POLYGON ((118.80055 31.87244, 118.80093 31.876...


,net_id,building_pop
1930,g_20689,0.0
2348,g_21241,1.0
2382,g_21291,2.0
1829,g_20468,2.0
234,g_12936,3.0
...,...,...
3570,g_23050,NaN
3571,g_23051,NaN
3572,g_23130,NaN
3573,g_23131,NaN


In [26]:
def get_G(x):
    threshold = time_threshold
    if x <= threshold:
        G = (np.exp(-0.5 * (x/threshold)**2) - np.exp(-0.5)) / (1 - np.exp(-0.5))
        return G
    else:
        return 0

def get_Rj(x):
    x = x.reset_index()
    Sj = x['park_area'][0]

    dt = 0
    for i in range(len(x)):
        if x['duration'][i] > 1:
            vl = x['building_pop'][i] * get_G(x['duration'][i])
            dt = dt + vl
        else:
            dt = 0
    # print(dt)
    if dt == 0:
        return 0
    else:
        return Sj/dt
    
def get_A(x):
    x = x.reset_index()
    dt = 0
    for i in range(len(x)):
        vl = x['Rj'][i] * get_G(x['duration'][i])
        dt = dt + vl
    return dt

In [27]:
def get_accessibility(file_name, time_threshold, grid_name, col_name):
    od_nav = pd.read_csv(r'./data_to_cal_accessibility/{}.csv'.format(file_name))
    # print(od_nav.head())
    od_nav[col_name] = od_nav[col_name].apply(lambda x: f'g_{x}')
    
    # print(od_nav.head(1))
    area_pop = joblib.load('./od_request_data_buildings/area_pop.joblib')
    area = area_pop['area']
    pop = get_gird_pop(grid_name, col_name)
    
    res = od_nav.merge(area, on='park_id',how='left').\
                                merge(pop, on=col_name,how='left')
    
    # print(res.head(1))
    res = res.fillna(0)
    resx = res.sort_values(by=[col_name,'park_id','duration'])
    resx = resx.drop_duplicates(subset=[col_name,'park_id'],keep='first')
    park_vk = resx.groupby(by='park_id').apply(get_Rj).reset_index().fillna(0)
    park_vk.columns=['park_id','Rj']
    
    res2 = resx.copy()
    res2 = res2.merge(park_vk, on='park_id')
    
    res2_vl = res2.groupby(by=col_name).apply(get_A)
    res2_vl = res2_vl.reset_index()
    res2_vl.columns = [col_name,'access_val']
    # print(res2_vl.head(1))
    

    gridnn = gpd.read_file(f"./shp/use_bounds/nanjing_{grid_name}.shp")
    # gridnn = gridnn[['net_id', 'building_pop']]
    gridnn[col_name] = gridnn[col_name].apply(lambda x: f'g_{x}')
    
    gridnn = gridnn.merge(res2_vl)
    print(gridnn.head(1))
    
    gridnn.to_file(f'./data_access_geojson/{file_name}_{time_threshold}.geojson')
    return gridnn

In [28]:
file_name

'od_nav_grid500_od'

In [29]:
gridnn = get_accessibility(file_name, time_threshold, grid_name, 'net_id') # net_id XZQDM

   net_id  building_pop
0   11925          1075
1   11926          3905
2   12294           536
3   12297           777
4   12383          6030
   net_id     Id                                           geometry
0   11834  south  POLYGON ((118.7796 31.87374, 118.77998 31.8782...
1   11835  south  POLYGON ((118.78484 31.87341, 118.78522 31.877...
2   11836  south  POLYGON ((118.79008 31.87309, 118.79046 31.877...
3   11837  south  POLYGON ((118.79531 31.87277, 118.79569 31.877...
4   11838  south  POLYGON ((118.80055 31.87244, 118.80093 31.876...
    net_id     Id                                           geometry  \
0  g_11834  south  POLYGON ((118.7796 31.87374, 118.77998 31.8782...   

   access_val  
0         0.0  


In [30]:
gridnn

,net_id,Id,geometry,access_val
0,g_11834,south,"POLYGON ((118.7796 31.87374, 118.77998 31.8782...",0.0
1,g_11835,south,"POLYGON ((118.78484 31.87341, 118.78522 31.877...",0.0
2,g_11836,south,"POLYGON ((118.79008 31.87309, 118.79046 31.877...",0.0
3,g_11837,south,"POLYGON ((118.79531 31.87277, 118.79569 31.877...",0.0
4,g_11838,south,"POLYGON ((118.80055 31.87244, 118.80093 31.876...",0.0
...,...,...,...,...
3570,g_23050,north,"POLYGON ((118.74853 32.25305, 118.74891 32.257...",0.0
3571,g_23051,north,"POLYGON ((118.75379 32.25272, 118.75417 32.257...",0.0
3572,g_23130,north,"POLYGON ((118.74365 32.25784, 118.74404 32.262...",0.0
3573,g_23131,north,"POLYGON ((118.74891 32.25751, 118.7493 32.2619...",0.0


# all accessibility

In [31]:
grid_types

['grid250',
 'grid500',
 'grid1000',
 'grid1500',
 'grid2000',
 'grid2500',
 'grid3000',
 'community']

In [32]:
trans_types = [
    'walk','cycle','pt',
    'nav']
time_thresholds = [900, 1800]


for time_threshold in time_thresholds:
    for trans_type in trans_types:
        for grid_name in grid_types:
            print(time_threshold, trans_type, grid_name)

            file_name = f'od_{trans_type}_{grid_name}_od'
                
            if grid_name == 'community':
                col_name= 'XZQDM' # XZQDM net_id
            else:
                col_name= 'net_id' # XZQDM net_id
            get_accessibility(file_name, time_threshold, grid_name, col_name)

900 walk grid250
   net_id  building_pop
0   46243           344
1   46244          3522
2   46417           731
3   46418           383
4   47686           136
   net_id     Id                                           geometry
0   46062  south  POLYGON ((118.77979 31.87597, 118.77998 31.878...
1   46063  south  POLYGON ((118.78241 31.87581, 118.7826 31.8780...
2   46064  south  POLYGON ((118.78503 31.87565, 118.78522 31.877...
3   46065  south  POLYGON ((118.78765 31.87549, 118.78784 31.877...
4   46066  south  POLYGON ((118.79027 31.87533, 118.79046 31.877...
    net_id     Id                                           geometry  \
0  g_46062  south  POLYGON ((118.77979 31.87597, 118.77998 31.878...   

   access_val  
0         0.0  
900 walk grid500
   net_id  building_pop
0   11925          1075
1   11926          3905
2   12294           536
3   12297           777
4   12383          6030
   net_id     Id                                           geometry
0   11834  south  POLYGON